# Problem 1: POTUS Tweet generator #

The goal of this task is to learn about the generation of text fragments by utilizing probabilistic language models. For this task, we provide you with a data set of scraped Twitter messages and will generate unique sentences by computing probabilities of N-grams. N-grams are sequences of $N$ consecutive words. For $N=2$ and the given text `will leave florida`, the n-grams would be `['will', 'leave'], ['leave','florida']`. This notebook includes questions for self-study which are not required to be answered but deepen the understanding of language processing.

### Learning Outcomes

- Generating n-grams from a given corpus
- Calculate the frequency of a successing word

### Passing criteria

When pushing the notebook to Artemis, all tests have to succeed. Do not change any function names or arguments.

<div class="alert alert-danger">
    <h3>DISCLAIMER</h3>
    <p>For the sake of testing and grading, we want to state that you are not allowed to import any other libraries and should not change the structure of the provided functions (i.a. the arguments and the name of the functions). Further, please make sure to use Python version 3.6 or higher.</p>
</div>

## Import

In [1]:
import re
import random
from helpers import cleanup_result

## 1. Loading the dataset (nothing to do here)

In order to generate twitter messages, it is essential to work on real messages. For this purpose, we provide you with scraped tweets from [@realDonaldTrump](https://twitter.com/realDonaldTrump). These tweets have been additionally processed by removing any clutter (images, links etc.) and retweeted messages. 
In the first step, the saved messages are therefore imported from the `.txt` file and splitted into an array of words.

In [2]:
def load_data(filename):
    """Load the tweet data as string and splitting it into words

    :param filename: Path to the data
    :returns data: Tweet data as a string

    """

    with open(filename, 'r') as file:
        data = file.read()
        data = ' '.join(data.split()).lower()
        return data

data = load_data("../data/data.txt")

In order top get a better grasp about the data set we just imported, the data is visualized in a wordcloud that shows words bigger if they tend to appear more often. The graphic below was created by this [generator](https://github.com/amueller/word_cloud).
<img src="assets/masked_wordcloud.png" alt="drawing" width="500"/>

## 2. Generate N-Grams
N-grams are sequences of $ n $ words, which need to be extracted from the text first. In this task you have to split up the given dataset into a list of lists. Each sublist should have length of `len(sublist) = N` which is provided through the function call.

Example (N=1):
```python
data = 'donald trump'
ngrams = generate_ngram_successors(data, 1)

print(ngrams)
```

This should return:
```bash
[['donald'], ['trump']]
```

In [3]:
def generate_ngram_successors(text, N):
    """Splitting the .txt file into a list of sublists of size N (N-Grams)

    :param text: Parsed twitter messages as String
    :param N: ngram parameter
    :returns ngram_successors: Dict of sublists of size N

    """

    # Initialize ngram_successors as a list
    ngrams = []
    
    # TODO: ['i', 'will', 'be', 'leaving', 'florida']
    # Splitting up the input file into a list of strings
    words = text.split(" ")
    
    # TODO: [['i', 'will', 'be'], ['will', 'be', 'leaving'], ['be', 'leaving', 'florida']] with N=3
    # Splitting all words into sublists of length N
    # and appending these to ngram_successors
    for i in range(len(words) - N + 1):
        ngrams.append(words[i : i + N])
    
    return ngrams

################################### UNCOMMENT FOR TESTING ###################################
# data = 'donald trump'
# ngrams = generate_ngram_successors(data, 1)
# print(ngrams)
#############################################################################################

## 3. Calculate the frequencies

This function should return a hash map that maps a given n-gram to the respective n-gram successor and its count.

Example (N=3):
```python
ngrams = [['will', 'leave', 'florida'], ['will', 'leave', 'nyc'], ['will', 'leave', 'florida']]
freqs = calculate_ngram_freqs(ngrams)

print(freqs)
```

This should return:
```bash
{'will leave': {'florida': 2, 'nyc': 1}}
```

In [4]:
def calculate_ngram_freqs(ngrams):
    """Calculate the frequency of a subsequent word given a sequence of ngrams

    :param ngrams: List of ngrams
    :returns freqs: Dict of frequency of successors
    
    """

    freqs = {}

    for ngram in ngrams:
        # !TODO¡
        # Differentiate successor (= lastWord) ...
        successor = ngram[-1]

        # TODO
        # ... from the ngram sequence
        ngram_seq  = " ".join(ngram[:-1])

        # If a given sequence is not known, initialize 
        # an empty hash map for ngram_seq.
        if ngram_seq not in freqs:
            freqs[ngram_seq] = {}

        # If successor not seen, set th count for 
        # the successor and the given sequence to 0.
        if successor not in freqs[ngram_seq]:
            freqs[ngram_seq][successor] = 0
        
        # TODO 
        # Increase frequency of successor given the sequence
        freqs[ngram_seq][successor] += 1

    return freqs


######################################## UNCOMMENT FOR TESTING ########################################
# test_gram = [['will', 'leave', 'florida'], ['will', 'leave', 'nyc'], ['will', 'leave', 'florida']]
# print(calculate_ngram_freqs(test_gram))
#######################################################################################################

## 4. Most probable successor

By finishing the previous function, we are actually able to caluclate the frequency of a successor for a given start sequence of `N-1` words. In case of Bigrams (N=3), this would mean that we would need two words to generate text. We want to test this now with the following function. Please complete the subsequent `next_word_max()` function and return the successor with the highest frequency among the possible successors.

In [5]:
def next_word_max(prev_seq, counts, N):
    """Returns the word with the highest occurence given a word sequence

    :param prev_seq: Previous sentence sequence
    :param counts: Respective frequency of successing words
    :param N: ngram parameter
    :returns max_successor: Next successor of given prev_seq

    """

    token_seq = " ".join(prev_seq.split()[-(N-1):])

    # TODO
    max_successor = max(counts[token_seq].items(), key=lambda k: k[1])[0]

    return max_successor

<div class="alert alert-info">
    <h3>Question</h3>
    <p>Do you think that it is a good idea to use the implemented function to generate randomized texts? Why?</p>
</div>

## 5. Sanity Check (nothing to do here)

You do not have to write any code here. With this step you only can ensure that your previous implementations work in the intended way. If you have done everything correct, this function should return you a message from the president.
By changing the String of `start_seq`, you affect the outcome of the function and can test for yourself the most frequent successor of a given sequence. Beware to increase the parameter `N` accordingly if you test a longer start sequence.

In [6]:
def sanity_check(data, start_seq="i am"):
    """Checks the previous function implementations
    
    :param data: Loaded twitter messages
    :param start_seq: Start sequence (Default value = "i am")
    :returns generated_text: Cleaned up message as a String
    
    """
    
    N = 3
    sentences = 0
    
    ngrams = generate_ngram_successors(data, N)
    counts = calculate_ngram_freqs(ngrams)
    
    generated_text = start_seq.lower()
    
    while sentences < 1:
        generated_text += " " + next_word_max(generated_text, counts, N)
        sentences += 1 if generated_text.endswith(('.','!', '?')) else 0
    
    generated_text = cleanup_result(generated_text)
    
    return generated_text


print(sanity_check(data))

I am very proud of you! 


## 6. Weighted successors

Spoiler Alert! You might have guessed it already: It might not be the best idea to only append the successor with the highest frequency to a given word or sequence of words. If we would do this everytime, we would always get back the same sentence for a given start sequence. To circumvent this issue and to be able to generate different messages with every function call, the objective of the next function is to choose out of all possible successors and taking the respective frequency into account.

In [7]:
def next_word_weighted(prev_seq, N, counts, seed=None):
    """Outputs the next word by taking the a weighted choice of the most recent tokens

    :param prev_seq: Previous sentence sequence
    :param N: ngram parameter
    :param counts: Respective frequency of successing words
    :returns next_word: Next successor of given prev_seq
    
    """

    token_seq = " ".join(prev_seq.split()[-(N-1):])
    choices = counts[token_seq].items()

    total = sum(weight for choice, weight in choices)
    
    successors, frequencies = zip(*list(choices))

    # TODO
    next_word = random.choices(population=successors, weights=[f / total for f in frequencies], k=1)

    return next_word[0]

## 7. Generate text messages

In [8]:
def gen_sentence(data, N=3):
    """Generate a random sentence based on text input file

    :param data: Loaded twitter messages
    :param N: ngram parameter (Default value = 3)
    :returns generated_text: Cleaned up message as a String

    """

    # Generate successors and calculate frequencies
    ngrams = generate_ngram_successors(data, N)
    counts = calculate_ngram_freqs(ngrams)

    # Randomize start sequence and amount of sentences
    sentence_count = random.randint(2, 5)
    start_seq = random.choice(list(counts))
    generated_text = start_seq.lower()
    sentences = 0

    while sentences < sentence_count:
        generated_text += " " + next_word_weighted(generated_text, N, counts)
        sentences += 1 if generated_text.endswith(('.','!', '?')) else 0

    return cleanup_result(generated_text)

In [9]:
if __name__ == "__main__":
    print(gen_sentence(data))

Past monday at 7:30 pm. Hes saddled our children and people like robert mueller, and around the world. 


<div class="alert alert-info">
    <h3>Questions</h3>
    <p>1. How could this model further be advanced to produce even better results?</p>
    <p>2. Play around with the window size N. What is the outcome? How does it affect the output?</p>
</div>